In [2]:
import pandas as pd
from modules.lavenshtein import lavenshtein

In [77]:
import os

def matching(user_str, text_col):
    laven_lst = []
    for i in range(len(text_col)):
        char_count = 0
        temp_laven = 0
        j = i
        temp_text = ""
        while (char_count < len(user_str)) and j < len(text_col):
            temp_text+=text_col.iloc[j]
            char_count += len(text_col.iloc[j])
            j+=1
        temp_laven = lavenshtein(temp_text,user_str)
        laven_lst.append(temp_laven/len(user_str))
    return min(laven_lst)

    
def get_laven(value):
    columns = ["BB1", "BB2", "BB3", "BB4", "BB5", "BB6", "BB7", "BB8", "Text_Main"]
    columns_labels = ["doc_id1","lv1","lp1","ld1","la1",
                            "doc_id2","lv2","lp2","ld2","la2",
                            "doc_id3","lv3","lp3","ld3","la3",
                            "doc_id4","lv4","lp4","ld4","la4",
                            "doc_id5","lv5","lp5","ld5","la5"]
    ocrdir = 'data/interim/ocr/'
    levan_user = pd.DataFrame(columns=["doc_id","levan_name","levan_price","levan_date","levan_address"])
    #filename = "00d0624439175.csv"
    for filename in os.listdir(ocrdir):
        ocr_temp = pd.read_csv(os.path.join(ocrdir, filename), header=None, names=columns)  
        text_col = ocr_temp["Text_Main"]
        levan_name = matching(value["vendor_name"],text_col)
        levan_price = matching(str(value["amount"]),text_col)
        levan_date = matching(value["date"],text_col)
        levan_address = matching(value["vendor_address"],text_col)
        levan_user.loc[levan_user.shape[0]]=[filename[:-4],levan_name,levan_price,levan_date,levan_address]
        
    smallest = levan_user.nsmallest(5, columns="levan_name")
    result = pd.Series(smallest.values.ravel(), name=0, index=columns_labels).T
    
    return result

users_data = pd.read_csv("data/original/Users.csv")
final_df = pd.DataFrame(columns=["doc_id1","lv1","lp1","ld1","la1"
                            "doc_id2","lv2","lp2","ld2","la2",
                            "doc_id3","lv3","lp3","ld3","la3",
                            "doc_id4","lv4","lp4","ld4","la4",
                            "doc_id5","lv5","lp5","ld5","la5"])
result=((users_data.apply(get_laven, axis=1)))




In [82]:
result["user_id"] = users_data["documentid"]
result.to_csv(os.path.join('data/final/', "final.csv"))

In [81]:
print(result)

         doc_id1  lv1  lp1  ld1  la1        doc_id2  lv2  lp2  ld2  la2  ...  \
0  00d0624439175    2    3    8   31  00d0876488640    2    4    8   31  ...   

  lv4  lp4  ld4  la4        doc_id5 lv5  lp5  ld5  la5        user_id  
0   2    3    7   25  00d0202365816   2    2    7   24  00d0624439175  

[1 rows x 26 columns]
